# Pas classificatie model /algorithme toe op  meldingen

Dit classificatie algorithme is een broncode geinspireerd op code van https://gitlab.com/commondatafactory/experiments/signalen-classificatie.


In [32]:
modelfile = 'model-db-bor-meld.pkl'
datafile = 'converted-mor-data-ods.csv'
basicresultfile = 'basic-classified-data.csv'
extendresultfile = 'extended-classified-data.csv'


In [33]:
import warnings

from nltk.stem.snowball import DutchStemmer
import nltk
#nltk.download('stopwords')
# init stemmer
stemmer=DutchStemmer(ignore_stopwords=True)

stop_words = set(nltk.corpus.stopwords.words('dutch'))
#print(stop_words)

import re
re_userid = re.compile("^([a-z]+\\:)")
def simple_preprocessor(text):
    text=text.lower()
    # text=re.sub(re_userid," ", text) # remove user name id
    text=re.sub("\\W"," ",text) # remove special chars
    
    # stem words
    words=re.split("\\s+",text)
    stemmed_words=[stemmer.stem(word=word) for word in words]
    return ' '.join(stemmed_words)

#warnings.simplefilter(action='ignore', category=FutureWarning)

In [34]:
# from sklearn.externals import joblib
import joblib

model = joblib.load(modelfile) 
class_labels = model.classes_
print('model loaded')
#print(len(class_labels))

# demodata

demo_sentence = ['Ik heb zonnepanelen en zou graag zien dat de gemeente de bomen voor ons huis flink snoeit of weghaalt']
demo_texts = [
'afval naast container gedumpt',
'Koelkast aan de kant van de weg',
'Auto met kenteken ZG-378F staat al maanden op een parkeerplaats \
in plaats van naar de Nassualaan te rijden',
'Leuk hoor al dat gesnoei maar kan de gemeente dan ook het snoeiafval meenemen\
en dit niet maanden laten liggen op de parkeerplaats',
'Mijn buurman heeft illegaal snoeiafval in de bosjes gedumpt samen met een oude fiets',
'Mijn buurman heeft illegaal in de bosjes gedumpt  een oude fiets',
'Ik heb zonnepanelen en zou graag zien dat de gemeente de bomen voor ons huis flink snoeit of weghaalt',
'In de sloot drijft een dode zwaan',
    'In de vijver met water cq gracht of sloot drijft een dode zwaan',
'Er ligt een dode meeuw op straat' ]



model loaded


# Demo

Demonstrate model by showing possible results based on demo inputs

In [35]:
import numpy as np
import pandas as pd
class_labels = model.classes_
print(*demo_sentence , "\n")
#print(model.predict_log_proba(demo_sentence))

y_pred_proba = model.predict_proba(demo_sentence) 

#dit is het eerste voorspelde item
y_pred_proba_item = y_pred_proba[0]

#dit zijn de regressie waarden van de  classe van dit item
#print (y_pred_proba_item)

#maak top n voor elke geclassificeerde item (i=1)
n = 3
top_n_pred = np.argsort(y_pred_proba, axis=1)[:,-n :]

#dit is de hele set met top n's per item i
print(top_n_pred)

# we printen de labels hulde
print(class_labels[top_n_pred])

#nu nog de regressie waarde
print(y_pred_proba_item[top_n_pred])


Ik heb zonnepanelen en zou graag zien dat de gemeente de bomen voor ons huis flink snoeit of weghaalt 

[[67  4 14]]
[['Verharding en civiele constructies  - Trillingen en geluid  - Trillingen bij gebruik verharding'
  'Afval  - Grofvuil  - Grofvuil niet aangeboden'
  'Groen  - Boomoverlast en boomschade  - Boomoverlast (te groot, te veel schaduw)']]
[[0.06272713 0.09217629 0.28301133]]


In [36]:

demo_preds = model.predict(demo_texts)
tuples = zip(demo_texts, demo_preds)
print(*('\"{1[0]}\"\n ----> {1[1]} \n'.format(*k) for k in enumerate(tuples)), sep="\n")


"afval naast container gedumpt"
 ----> Afval  - Afvalcontainer  - Vuil naast container 

"Koelkast aan de kant van de weg"
 ----> Afval  - Grofvuil  - Grofvuil niet aangeboden 

"Auto met kenteken ZG-378F staat al maanden op een parkeerplaats in plaats van naar de Nassualaan te rijden"
 ----> Parkeren  - Parkeren voertuigen  - Overig foutparkeren 

"Leuk hoor al dat gesnoei maar kan de gemeente dan ook het snoeiafval meenemenen dit niet maanden laten liggen op de parkeerplaats"
 ----> Groen  - Struiken: snoeien en schade  - Snoeiafval niet verwijderd 

"Mijn buurman heeft illegaal snoeiafval in de bosjes gedumpt samen met een oude fiets"
 ----> Groen  - Struiken: snoeien en schade  - Snoeiafval niet verwijderd 

"Mijn buurman heeft illegaal in de bosjes gedumpt  een oude fiets"
 ----> Afval  - Overig afval / lozingen  - Fietswrak 

"Ik heb zonnepanelen en zou graag zien dat de gemeente de bomen voor ons huis flink snoeit of weghaalt"
 ----> Groen  - Boomoverlast en boomschade  - Boomov

# Basic Prediction for inputfiles
Drops empty lines which cant be classified are dropped
single class no proba

In [37]:
import pandas as pd
df = pd.read_csv(datafile, engine='python')
df.head()

print(len(df),'rows import as job') 
df = df.dropna(axis=0, how='any', thresh=None, subset=['omschrijving'], inplace=False)
inputs = df["omschrijving"]

predicted = model.predict(inputs)
df['predicted'] = predicted

# save to file
print(len(df),'rows classified exported as result') 
df.to_csv(basicresultfile, index=False )



77930 rows import as job
72899 rows classified exported as result


# Extended prediction for inputfiles
return s 
p1,p2,p3 with probabilities
research into how many p's are required 

In [38]:
import pandas as pd
df = pd.read_csv(datafile, engine='python')
print(len(df),'rows import as job') 
df = df.dropna(axis=0, how='any', thresh=None, subset=['omschrijving'], inplace=False)
inputs = df["omschrijving"]

#change to inputs for production
y_pred_proba = model.predict_proba(inputs)

#maak top n voor elke geclassificeerde item (i=1)
n = 3
top_n_pred = np.argsort(y_pred_proba, axis=1)[:,-n :]

#dit is de hele set met top n's index voor alle texts
#print(top_n_pred)

# top_n classes als string voor alle texts
top_n_class = class_labels[top_n_pred]
#print(top_n_class)

#nu nog de regressie waarden van top n voor alle texts
top_n_proba = np.sort(y_pred_proba, axis=1)[:,-n :]
#print(top_n_proba)

#SAVING THE DATA TO DISK
#use numpy array to select 1,2,3 th value of n to add to data file
print(" ")
for i in range(n): 
    df[('p_class_' + str(n-i))] = top_n_class[:,i]
    df[('p_proba_' + str(n-i))] = top_n_proba[:,i]

# save to file
print(len(df),'rows extend classified exported as result') 
df.to_csv(extendresultfile, index=False )

77930 rows import as job
 
72899 rows extend classified exported as result
